In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle
from astropy.visualization import make_lupton_rgb
from functions.plot_data import plot_data
import glob
from PIL import Image
from pathlib import Path
import shutil

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, _ = load_pickles(select="stokes")

In [ ]:
# %%capture
# # Generate gifs of evolution through all frames

# for param in stokes_list:
#     # Make temp folder for frames
#     Path("generated/temp").mkdir(parents=True, exist_ok=True)

#     # Generate frames
#     for frame_index in range(18,29,2):
#     # for frame_index in range(4):
#         fig = stokes_list[param].plot_frame_n(frame_index)
#         fig.savefig(f"generated/temp/{frame_index:03}.png", dpi=100)

#     # Save frames to gif
#     frames = [Image.open(image) for image in glob.glob('./generated/temp/*.png')]
#     frame_one = frames[0]
#     gifname = "generated/" + str(param) + "_all_gif.gif"
#     print("Saved figure to file", gifname)
#     frame_one.save(gifname, format="GIF", append_images=frames, save_all=True, duration=150, loop=0)

#     # Remove temp folder
#     shutil.rmtree("./generated/temp/")

In [ ]:
# %%capture
# # Generate images of all frames

# for param in stokes_list:
#     # Make temp folder for frames
#     Path("generated/" + str(param) + "temp").mkdir(parents=True, exist_ok=True)

#     # Generate frames
#     for frame_index in range(stokes_list[param].size_wave):
#     # for frame_index in range(4):
#         fig = stokes_list[param].plot_frame_n(frame_index)
#         fig.savefig("generated/" + str(param) + "temp/" + str(frame_index) + ".png", dpi=200)

In [ ]:
# Plot the spectrum with calibrated wavelengths, adding vertical lines to represent the four points selected for representation
plt.figure(figsize=(7,2.5))
plt.plot(stokes_list['I'].wave_array[4:53], stokes_list['I'].mean_quiet[4:53] / 10280.54, color='purple')
plt.xlabel('Wavelength [' + r'$\AA$' + ']')
plt.ylabel(r'$ I\:/\:I_{0,QS}$')


# Select 4 frames leading up to the absorption line (each frame represents rising through the photosphere)
selected_frames = [12, 24, 27, 29]
plt.vlines(stokes_list['I'].wave_array[selected_frames], stokes_list['I'].mean_quiet.min() / 10280.54, stokes_list['I'].mean_quiet.max() / 10280.54, colors='red', ls='--', lw=1,)
plt.show()

In [ ]:
# Plot original frames along spectral line
for param in stokes_list:
    fig, axs = plt.subplots(2, 2, figsize=(10,10))
    for i in range(2):
        for j in range(2):
            img = axs[i, j].imshow(stokes_list[param].data[:,:,selected_frames[i*2 + j]], cmap='gray')
            axs[i, j].set_title(f'{param} data at frame {selected_frames[i*2 + j]}')
            axs[i, j].get_xaxis().set_visible(False)
            axs[i, j].get_yaxis().set_visible(False)
            divider = make_axes_locatable(axs[i, j])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            fig.colorbar(img, cax=cax)


    fig.tight_layout()
    fig.savefig("generated/" + str(param) + "_4_frames_original.png", dpi=150)
    print("Saved figure to file", "generated/" + str(param) + "_4_frames_original.png")

In [ ]:
# Plot frames along spectral line
for param in stokes_list:
    fig, axs = plt.subplots(2, 2, figsize=(9,10))
    for i in range(2):
        for j in range(2):
            img = axs[i, j].imshow(stokes_list[param].data_n[:,:,selected_frames[i*2 + j]], cmap='gray')
            axs[i, j].set_title(f'{param} data at {stokes_list[param].wave_array[selected_frames[i*2 + j]]:.2f}')
            axs[i, j].label_outer()
            divider = make_axes_locatable(axs[i, j])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            fig.colorbar(img, cax=cax)

    for ax in axs.flat:
        ax.label_outer()


    fig.savefig("generated/" + str(param) + "_4_frames.png", dpi=150)
    print("Saved figure to file", "generated/" + str(param) + "_4_frames.png")
    fig.tight_layout()
    fig.savefig("generated/" + str(param) + "_4_frames_norm.png", dpi=150)
    print("Saved figure to file", "generated/" + str(param) + "_4_frames_norm.png")

In [ ]:

# Plot frames along spectral line
frame_index = 5
fig, axs = plt.subplots(2, 2, figsize=(9,10))
for param in stokes_list:
    for i in range(2):
        for j in range(2):
            img = axs[i, j].imshow(stokes_list[param].data_n[:,:,frame_index], cmap='mag')
            axs[i, j].set_title(f'{param} data in the continuum at {stokes_list[param].wave_array[frame_index]:.2f}')
            axs[i, j].label_outer()
            divider = make_axes_locatable(axs[i, j])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            fig.colorbar(img, cax=cax)

    for ax in axs.flat:
        ax.label_outer()


    fig.tight_layout()
    fig.savefig("generated/" + str(param) + "_4_frames_norm.png", dpi=150)
    print("Saved figure to file", "generated/" + str(param) + "_4_frames_norm.png")

In [ ]:
import matplotlib.colors as colors

class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin, vmax, midpoint=0, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        normalized_min = max(0, 1 / 2 * (1 - abs((self.midpoint - self.vmin) / (self.midpoint - self.vmax))))
        normalized_max = min(1, 1 / 2 * (1 + abs((self.vmax - self.midpoint) / (self.midpoint - self.vmin))))
        normalized_mid = 0.5
        x, y = [self.vmin, self.midpoint, self.vmax], [normalized_min, normalized_mid, normalized_max]
        return np.ma.masked_array(np.interp(value, x, y))

In [ ]:
# Plot frames along spectral line, for each Stokes parameter
fig, axs = plt.subplots(4, 4, figsize=(12,15))
i = 0
extent=[0, 89.88485, 0, 122.88]
norm = [0.3, 0.2, 0.4]
colourbar_label=[r'$ I\:/\:I_{0,QS}$', r'$ Q\:/\:I_{0,QS}$', r'$ U\:/\:I_{0,QS}$', r'$ V\:/\:I_{0,QS}$']
for param in stokes_list:
    if param == 'I':
        colourmap = 'magma'
        divnorm=None
        vmin=0
        vmax=1
    else:
        colourmap = 'PuOr_r'
        divnorm=MidpointNormalize(vmin=-norm[i-1], vmax=norm[i-1], midpoint=0)
        vmin=None
        vmax=None

    for frame_index in range(4):
        axs[i, frame_index].imshow(stokes_list[param].data_n[:,:,selected_frames[frame_index]], origin='lower', extent=extent, cmap=colourmap, norm=divnorm, vmin=vmin, vmax=vmax)
        axs[i, frame_index].set_title(f'{param} data at {stokes_list[param].wave_array[selected_frames[frame_index]]:.2f} ' + r'$\AA$')

        if frame_index != 0:
            axs[i, frame_index].set_yticklabels([])
            axs[i, frame_index].set_yticks([])
        else:
            axs[i, frame_index].set_ylabel('y [arcsec]')

        if param != 'V':
            axs[i, frame_index].set_xticklabels([])
            axs[i, frame_index].set_xticks([])
        else:
            axs[i, frame_index].set_xlabel('x [arcsec]')

    if frame_index == 3:
        divider = make_axes_locatable(axs[i, frame_index])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(axs[i, frame_index].images[0], cax=cax, label=colourbar_label[i])



    i = i+1

fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=-0.5, hspace=0.1)

for ax in axs.flat:
    ax.label_outer()

fig.tight_layout()

fig.savefig("generated/Stokes_4_frames.png", dpi=150, bbox_inches='tight')
print("Saved figure to file", "generated/Stokes_4_frames.png")

In [ ]:
# Plot the spectrum with calibrated wavelengths, adding vertical lines to represent the four points selected for representation
plt.figure(figsize=(8,5))
plt.plot(stokes_list['I'].wave_array[60:110], stokes_list['I'].mean_quiet[60:110] / 10280.54, color='purple')
plt.xlabel('Calibrated wavelength [' + r'$\AA$' + ']')
plt.xlabel('Calibrated wavelength [' + r'$\AA$' + ']')

# Select 4 frames leading up to the absorption line (each frame represents rising through the photosphere)
selected_frames = np.array([10, 24, 27, 29])+46

plt.vlines(stokes_list['I'].wave_array[selected_frames], stokes_list['I'].mean_quiet.min() / 10280.54, stokes_list['I'].mean_quiet.max() / 10280.54, colors='red', ls='--', lw=2,)
plt.show()

In [ ]:
# Plot frames along spectral line, for each Stokes parameter
fig, axs = plt.subplots(4, 4, figsize=(12,15))
i = 0
extent=[0, 89.88485, 0, 122.88]
norm = [0.3, 0.2, 0.4]
colourbar_label=[r'$ I\:/\:I_{0,QS}$', r'$ Q\:/\:I_{0,QS}$', r'$ U\:/\:I_{0,QS}$', r'$ V\:/\:I_{0,QS}$']
for param in stokes_list:
    if param == 'I':
        colourmap = 'magma'
        divnorm=None
        vmin=0
        vmax=1
    else:
        colourmap = 'PuOr_r'
        divnorm=MidpointNormalize(vmin=-norm[i-1], vmax=norm[i-1], midpoint=0)
        vmin=None
        vmax=None

    for frame_index in range(4):
        axs[i, frame_index].imshow(stokes_list[param].data_n[:,:,selected_frames[frame_index]], origin='lower', extent=extent, cmap=colourmap, norm=divnorm, vmin=vmin, vmax=vmax)
        axs[i, frame_index].set_title(f'{param} data at {stokes_list[param].wave_array[selected_frames[frame_index]]:.2f} ' + r'$\AA$')

        if frame_index != 0:
            axs[i, frame_index].set_yticklabels([])
            axs[i, frame_index].set_yticks([])
        else:
            axs[i, frame_index].set_ylabel('y [arcsec]')

        if param != 'V':
            axs[i, frame_index].set_xticklabels([])
            axs[i, frame_index].set_xticks([])
        else:
            axs[i, frame_index].set_xlabel('x [arcsec]')

    if frame_index == 3:
        divider = make_axes_locatable(axs[i, frame_index])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(axs[i, frame_index].images[0], cax=cax, label=colourbar_label[i])



    i = i+1

fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=-0.5, hspace=0.1)

for ax in axs.flat:
    ax.label_outer()

fig.tight_layout()

fig.savefig("generated/Stokes_4_frames_second.png", dpi=150, bbox_inches='tight')
print("Saved figure to file", "generated/Stokes_4_frames_second.png")